In [ ]:
from grid.robot.aerial.airgen_drone import AirGenDrone 
airgen_drone_0 = AirGenDrone()

In [ ]:
from grid.utils.types import Position
airgen_drone_0.takeoff()
airgen_drone_0.moveToPosition(Position(0, 0, -25))

In [ ]:
import airgen
import numpy as np
from grid.model.perception.detection.gdino import GroundingDINO

groundingdino = GroundingDINO()

# Define a detect function that captures an RGB image from the front camera and uses an object detector
# Please seperate the object name with a dot (.)

def detect(drone, object_name="fire."):
    rgb_image = drone.getImage("front_center", image_type="rgb")
    boxes, scores, labels = groundingdino.run(rgb_image.data, object_name)
    return boxes, scores, labels

# Let's search for the fire by rotating 360 degrees and trying to detect fire at each step
yaw_angles = np.linspace(0, 360, 30)

for yaw in yaw_angles:
    airgen_drone_0.setYaw(np.deg2rad(yaw))
    boxes, scores, labels = detect(airgen_drone_0)
    if 'fire' in labels:
        break

In [ ]:
# Let's try to use a different model and object category

from grid.model.perception.segmentation.clipseg import CLIPSeg
clipseg = CLIPSeg(use_local = True)

# Use CLIP features to try and segment out 'smoke'

def segment(drone, object_name="smoke"):
    rgb_image = drone.getImage("front_center", image_type="rgb")
    result = clipseg.run(rgb_image.data, object_name)
    return result

segment(airgen_drone_0)

In [ ]:
# Let's evaluate the vision models with some weather variations to the scene
import time
airgen_drone_0.client.simEnableWeather(True)

for i in range(10):
    airgen_drone_0.client.simSetWeatherParameter(airgen.WeatherParameter.Rain, i / 10)
    airgen_drone_0.client.simSetWeatherParameter(airgen.WeatherParameter.Fog, i / 10)
    detect(airgen_drone_0)
    segment(airgen_drone_0)
    time.sleep(1)